In [20]:
import os
pca_components = 500

In [21]:
# camera-ready version
subject_list = ["01","02","03","04","05","06","07","08","09"]
name_base_layer_key_list = [('50_0_6', 'layer_0'),  #lexicon
                            ('50_0_6', 'residual'), #syntax
                        ('50_6_20', 'residual'), #meaning
                        ('50_20_30', 'layer_30'),  #reasoning
                        ('50_20_30', 'residual')] #full

## shuffle

### slurm version

In [28]:
subject_list = ["01","02","03","04","05","06","07","08","09"]
for subj in subject_list:
    for name_base, layer_key in name_base_layer_key_list:
        sbatch_file_name = f"sf_{name_base}_{layer_key}_{subj}.sh"
        with open(sbatch_file_name, 'w', newline='\n') as f:
            f.write("#!/bin/bash\n")
            f.write(f"#SBATCH --job-name={subj}_{name_base}_{layer_key}\n")
            f.write(f"#SBATCH --output=log_{subj}_{name_base}_{layer_key}.txt\n")
            f.write(f"#SBATCH --error=err_{subj}_{name_base}_{layer_key}.txt\n")
            f.write("#SBATCH --mem=128G\n")
            f.write("#SBATCH --cpus-per-task=1\n")
            f.write("#SBATCH --partition=naplab,burst\n")
            f.write(f"python -u shuffle.py --name_base {name_base} --layer_key {layer_key} --subj {subj} --pca_components {pca_components}\n")

In [25]:
import os
from datetime import datetime

bash_file_name = "sf_good_layer.sh"

with open(bash_file_name, 'w', newline='\n') as f:
    f.write("#!/bin/bash\n")
    f.write(f"# Auto-generated at {datetime.now()}\n")
    f.write("set -e  # exit when any command fails\n\n")   #

    for subj in subject_list:
        for name_base, layer_key in name_base_layer_key_list:
            sbatch_file_name = f"./sf_{name_base}_{layer_key}_{subj}.sh"
            f.write(f"echo 'Submitting {sbatch_file_name}'\n")
            f.write(f"sbatch {sbatch_file_name}\n")

os.chmod(bash_file_name, 0o755)

print("Generated bash successfully!")

### jupyter version

In [ ]:
import subprocess
from pathlib import Path

# Run shuffle locally inside the notebook without submitting to Slurm.
log_dir = Path("./shuffle_logs")
log_dir.mkdir(exist_ok=True)

subject_list = ["01","02","03","04","05","06","07","08","09"]
for subj in subject_list:
    for name_base, layer_key in name_base_layer_key_list:
        log_file = log_dir / f"shuffle_{subj}_{name_base}_{layer_key}.txt"
        cmd = [
            "python","-u","shuffle.py",
            "--name_base", name_base,
            "--layer_key", layer_key,
            "--subj", subj,
            "--pca_components", str(pca_components)
        ]
        print(f"Running {' '.join(cmd)}")
        with log_file.open("w") as lf:
            subprocess.run(cmd, check=True, stdout=lf, stderr=subprocess.STDOUT)
        print(f"✔ Saved log to {log_file}")

## encoding

### slurm version

In [17]:
for pca_components in [500]:
    for name_base, layer_key in name_base_layer_key_list:
        sbatch_file_name = f"encoding_{name_base}_{layer_key}_{pca_components}.sh"
        with open(sbatch_file_name, 'w', newline='\n') as f:
            f.write("#!/bin/bash\n")
            f.write(f"#SBATCH --job-name=pt_{name_base}_{layer_key}_{pca_components}\n")
            f.write(f"#SBATCH --output=log_pt_{name_base}_{layer_key}_{pca_components}.txt\n")
            f.write(f"#SBATCH --error=err_pt_{name_base}_{layer_key}_{pca_components}.txt\n")
            f.write("#SBATCH --mem=128G\n")
            f.write("#SBATCH --cpus-per-task=1\n")
            f.write("#SBATCH --partition=naplab,burst\n")
            # f.write("cd /home/lh3288/project/PJ2504_Dual_stream/LLM_Dual_Stream/Brain_Encoding/Hasson\n")
            f.write(f"python -u encoding.py --name_base {name_base} --layer_key {layer_key} --pca_components {pca_components}\n")

In [19]:
import os
from datetime import datetime

bash_file_name = "encoding_all.sh"

with open(bash_file_name, 'w', newline='\n') as f:
    f.write("#!/bin/bash\n")
    f.write(f"# Auto-generated at {datetime.now()}\n")
    f.write("set -e  # exit when any command fails\n\n")   #

    for pca_components in [500]:
        for name_base, layer_key in name_base_layer_key_list:
            print(name_base, layer_key)
            sbatch_file_name = f"./scaled_{name_base}_{layer_key}_{pca_components}.sh"
            f.write(f"echo 'Submitting {sbatch_file_name}'\n")
            f.write(f"sbatch {sbatch_file_name}\n")

os.chmod(bash_file_name, 0o755)

print("Generated successfully!")

### jupyter version

In [ ]:
import subprocess
from pathlib import Path

# Run encoding locally; adjust `pca_values` for sweeps.
encoding_log_dir = Path("./encoding_logs")
encoding_log_dir.mkdir(exist_ok=True)

pca_values = [pca_components]  # reuse the global default defined above
for current_pca in pca_values:
    for name_base, layer_key in name_base_layer_key_list:
        log_file = encoding_log_dir / f"encoding_{name_base}_{layer_key}_{current_pca}.txt"
        cmd = [
            "python","-u","encoding.py",
            "--name_base", name_base,
            "--layer_key", layer_key,
            "--pca_components", str(current_pca)
        ]
        print(f"Running {' '.join(cmd)}")
        with log_file.open("w") as lf:
            subprocess.run(cmd, check=True, stdout=lf, stderr=subprocess.STDOUT)
        print(f"✔ Saved log to {log_file}")